In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from wordcloud import WordCloud

from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
training = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/train.csv')
test = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/test.csv')
test_labels = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/test_labels.csv')

In [ ]:
#training = pd.read_csv("C:/Users/Falak/Desktop/NLP/train.csv")
#test = pd.read_csv("C:/Users/Falak/Desktop/NLP/test.csv")
#test_labels = pd.read_csv("C:/Users/Falak/Desktop/NLP/test_labels.csv")

In [ ]:
#training = pd.read_csv('C:/Users/Alvis/STA142A/train.csv')
#test = pd.read_csv('C:/Users/Alvis/STA142A/test.csv')
#test_labels = pd.read_csv("C:/Users/Alvis/STA142A/test_labels.csv")

In [3]:
# Preprocessing
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def preprocess(training, test):
    training['clean'] = training.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1) #removes new line character
    test['clean'] = test.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1)

    #removes urls
    training['clean'] = training.apply(lambda row: re.sub('http://\S+|https://\S+', '', row['clean']), axis=1)
    test['clean'] = test.apply(lambda row: re.sub('http://\S+|https://\S+', '',row['clean']), axis=1)

    #remove all non-alphanumeric values
    training['clean'] = training.apply(lambda row: re.sub('[^A-Za-z0-9\' ]+', '',row['clean']), axis=1)
    test['clean'] = test.apply(lambda row: re.sub('[^A-Za-z0-9\' ]+', '',row['clean']), axis=1)

    #remove stopwords as they occupy major chunk of the vocabulary
    training['clean'] = training['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word in (stop)]))
    test['clean'] = test['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word in (stop)]))

    #removes all additional spaces
    training['clean'] = training.apply(lambda row: re.sub(' +', ' ', row['clean']).strip(), axis=1)
    test['clean'] = test.apply(lambda row: re.sub(' +', ' ', row['clean']).strip(), axis=1)
    
    #lowercase end result
    training['clean'] = training['clean'].str.lower()
    test = test['clean'].str.lower()

preprocess(training, test)
training.head(10)
test.head(10)

,id,comment_text,clean
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,Yo bitch Ja Rule succesful ever whats hating s...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,From RfC The title fine IMO
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",Sources Zawe Ashton Lapland
3,00017563c3f7919a,":If you have a look back at the source, the in...",If look back source information I updated corr...
4,00017695ad8997eb,I don't anonymously edit articles at all.,I anonymously edit articles
5,0001ea8717f6de06,Thank you for understanding. I think very high...,Thank understanding I think highly would rever...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...,Please add nonsense Wikipedia Such edits consi...
7,000247e83dcc1211,:Dear god this site is horrible.,Dear god site horrible
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ...",Only fool believe numbers The correct number l...
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...,Double Redirects When fixing double redirects ...


In [ ]:
# Without k-fold cross validation
# for loop to check the accuracy score for each label

for i in range(2,8):
    X_train, X_validation = train_test_split(training, random_state = 999) #default is 25% split for validation
    Y_train = X_train.iloc[:,i]

    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
    tfidf_vectorizer.fit_transform(X_train['clean'].values)

    train_feature_set=tfidf_vectorizer.transform(X_train['clean'].values)
    test_feature_set=tfidf_vectorizer.transform(X_validation['clean'].values)

    scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
    model = scikit_log_reg.fit(train_feature_set, Y_train) #Model build using training data cleaned comments

    Y_predicted = model.predict(test_feature_set) #Y_predicted are the predicted labels from our model
    Y_predicted_prob = model.predict_proba(test_feature_set) #proba model shows two columns: 1st column is prob that it is classified as 0, 2nd column is prob that it is classified as 1
    
    #Predicted Accuracy score

    Y_validation = X_validation.iloc[:,i]
    print(f"The classification accuracy of the fitted model for the {training.columns[i]} label is {accuracy_score(Y_validation, Y_predicted)}")

In [54]:
#k fold crossvalidation
from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

kf2 = KFold(n_splits = 2)
kf = KFold(n_splits = 5)
kf10 = KFold(n_splits = 10)

#comments -> numbers
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
tfidf_vectorizer.fit_transform(training['clean'].values)

#cleaned complete training set
training_feature_set = tfidf_vectorizer.transform(training['clean'].values)

scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
print(training['toxic'][6])

In [12]:
import sys, os
def LogRegFit_NoMessage(X, Y, logreg_obj):
    old_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    
    model = logreg_obj.fit(X, Y) #Model build using training data cleaned comments
    
    sys.stdout = old_stdout
    
    return(model)

In [13]:


#applied cross validation on one class 'toxic'
MSE = []
for train_index, valid_index in kf10.split(training_feature_set):
    X_train, X_valid = training_feature_set[train_index], training_feature_set[valid_index]
    Y_train, Y_valid = training['toxic'][train_index], training['toxic'][valid_index]
    
    #Model build using training data cleaned comments
    model = LogRegFit_NoMessage(X_train, Y_train, scikit_log_reg)
    
    # predicted y labels using X validation set
    Y_predicted = model.predict(X_valid)
    #Y_predicted = model.predict_proba(X_valid)
    score = accuracy_score(Y_valid, Y_predicted)
    
    MSE.append(score)
    
np.mean(MSE)

0.9593848431156665

In [14]:
# takes in feature set and class, returns accuracy score of logistic regression model for that classs
def calcClassMSE(FeatureSet, Class):
    MSE = []
    
    for train_index, valid_index in kf10.split(training_feature_set):
        X_train, X_valid = training_feature_set[train_index], training_feature_set[valid_index]
        Y_train, Y_valid = training[Class][train_index], training[Class][valid_index] #labels

        #Model build using training data cleaned comments
        model = LogRegFit_NoMessage(X_train, Y_train, scikit_log_reg) 

        # predicted y labels using X validation set
        Y_predicted = model.predict(X_valid)
        #Y_predicted = model.predict_proba(X_valid)
        score = accuracy_score(Y_valid, Y_predicted)

        MSE.append(score)

    return(np.mean(MSE))

calcClassMSE(training_feature_set, "toxic")

0.9593848431156665

In [ ]:
classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

mse_list = {"toxic":[],"severe_toxic":[],"obscene":[],"threat":[],"insult":[],"identity_hate":[]}

for c in classes:
    mse_list[c].append(calcClassMSE(training_feature_set, c))

In [24]:
mse_list

{'toxic': [0.9593848431156665],
 'severe_toxic': [0.9904870003945856],
 'obscene': [0.9784672678609574],
 'threat': [0.9972864762041469],
 'insult': [0.9712040429842066],
 'identity_hate': [0.992059954994193]}

In [25]:
max(mse_list, key = mse_list.get)

'threat'

In [ ]:
classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

mse_list = {"toxic":[],"severe_toxic":[],"obscene":[],"threat":[],"insult":[],"identity_hate":[]}

for c in classes:
    mse_list[c].append(calcClassProba(training_feature_set, c))

In [ ]:
# CountVectorizer implementation - same as TF-IDF, but returns ints instead of floats. Makes MSE slightly lower but just because of integer rounding most likely
from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

kf = KFold(n_splits = 5)
kf10 = KFold(n_splits = 10)

#comments -> numbers
cv = CountVectorizer(binary = True, max_df = 0.7)
cv.fit_transform(training['clean'].values)

#cleaned complete training set
training_feature_set_binary = cv.transform(training['clean'].values)

scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)

MSE = []
for train_index, valid_index in kf10.split(training_feature_set_binary):
    X_train, X_valid = training_feature_set_binary[train_index], training_feature_set_binary[valid_index]
    Y_train, Y_valid = training['toxic'][train_index], training['toxic'][valid_index]
    
    model = scikit_log_reg.fit(X_train, Y_train) #Model build using training data cleaned comments

    # predicted y labels using X validation set
    Y_predicted = model.predict(X_valid)
    #Y_predicted = model.predict_proba(X_valid)
    score = accuracy_score(Y_valid, Y_predicted)
    
    MSE.append(score)
 
np.mean(MSE)